# Cypher Query Development Notebook

This notebook provides an interactive environment for developing and testing Cypher queries against the EKG (Enhanced Knowledge Graph) database. It includes functionality for both raw Cypher queries and natural language to Cypher translation.

## Features
- Execute raw Cypher queries directly
- Convert natural language queries to Cypher automatically
- View query results in tabular format
- Access to graph schema documentation
- Comprehensive examples of common queries
- Query performance analysis
- Schema exploration tools

## Usage
1. Make sure the EKG database is populated with data
2. Use the cells below to experiment with different queries
3. Modify the examples to suit your needs

## Quick Start
- Run the setup cell below to initialize all functions
- Use `execute_cypher_query()` for raw Cypher queries
- Use `execute_text_query()` for natural language queries
- Use `show_schema()` to view the graph schema


In [1]:
# === SETUP AND INITIALIZATION ===

# Import required modules
import time

from rich.console import Console
from rich.panel import Panel
from rich.table import Table

# Import EKG modules
from genai_graph.core.graph_backend import create_backend_from_config
from genai_graph.core.graph_registry import GraphRegistry
from genai_graph.core.schema_doc_generator import generate_schema_description
from genai_graph.core.text2cypher import query_kg

# Initialize console for rich output
console = Console()

# Configuration
GRAPH_DB_CONFIG = "default"


# Helper function to display results in a table
def display_results(df, title="Query Results", max_rows=20):
    """Display query results in a rich table format."""
    if df is None or df.empty:
        console.print("[yellow]Query returned no results[/yellow]")
        return

    table = Table(title=f"{title} ({len(df)} rows)")
    for col in df.columns:
        table.add_column(str(col), style="cyan")

    # Limit rows for readability
    for i, (_, row) in enumerate(df.iterrows()):
        if i >= max_rows:
            table.add_row(*["..." for _ in df.columns])
            break
        table.add_row(*[str(val) for val in row])

    console.print(table)

    if len(df) > max_rows:
        console.print(f"[dim]Showing first {max_rows} of {len(df)} results[/dim]")


# Get database connection
def get_backend():
    """Get the graph backend connection."""
    backend = create_backend_from_config(GRAPH_DB_CONFIG)
    if not backend:
        raise Exception("No EKG database found")
    return backend


# Execute raw Cypher query with timing
def execute_cypher_query(cypher_query, show_timing=True):
    """Execute a raw Cypher query and display results."""
    if not cypher_query.strip():
        console.print("[red]❌ Empty query[/red]")
        return None

    try:
        console.print(Panel("[bold cyan]Querying EKG Database[/bold cyan]"))
        console.print(f"[dim]Executing: {cypher_query}[/dim]")

        start_time = time.time()
        backend = get_backend()
        result = backend.execute(cypher_query)
        df = result.get_as_df()
        execution_time = time.time() - start_time

        display_results(df)

        if show_timing:
            console.print(f"[dim]Query executed in {execution_time:.3f} seconds[/dim]")

        return df

    except Exception as e:
        console.print(f"[red]❌ Query error: {e}[/red]")
        return None


# Execute text-to-Cypher query with timing
def execute_text_query(natural_language_query, subgraphs=None, llm_id=None, show_timing=True):
    """Convert natural language to Cypher and execute the query."""
    if not natural_language_query.strip():
        console.print("[red]❌ Empty query[/red]")
        return None

    try:
        console.print(Panel("[bold cyan]Converting Text to Cypher[/bold cyan]"))
        console.print(f"[dim]Natural Language Query: {natural_language_query}[/dim]")

        start_time = time.time()
        # If no subgraphs are provided, use all registered ones
        registry = GraphRegistry.get_instance()
        selected_subgraphs = subgraphs or registry.listsubgraphs()

        df = query_kg(natural_language_query, subgraphs=selected_subgraphs, llm_id=llm_id)
        execution_time = time.time() - start_time

        if df.empty:
            console.print("[yellow]Query returned no results[/yellow]")
            return df

        display_results(df, "Query Results")

        if show_timing:
            console.print(f"[dim]Query executed in {execution_time:.3f} seconds[/dim]")

        return df

    except Exception as e:
        console.print(f"[red]❌ Query error: {e}[/red]")
        return None


# Display graph schema
def show_schema(subgraphs=None):
    """Display the graph schema documentation."""
    try:
        console.print(Panel("[bold cyan]Knowledge Graph Schema[/bold cyan]"))

        # If no subgraphs are provided, use all registered ones
        registry = GraphRegistry.get_instance()
        selected_subgraphs = subgraphs or registry.listsubgraphs()

        desc = generate_schema_description(selected_subgraphs)
        console.print(desc)

    except Exception as e:
        console.print(f"[red]❌ Error displaying schema: {e}[/red]")
        return None


# Get database statistics
def get_database_stats():
    """Get comprehensive database statistics."""
    try:
        backend = get_backend()
        console.print(Panel("[bold cyan]Database Statistics[/bold cyan]"))

        # Get table information
        tables_result = backend.execute("CALL show_tables() RETURN *")
        tables_df = tables_result.get_as_df()

        node_tables = []
        rel_tables = []

        for _, row in tables_df.iterrows():
            if row.get("type") == "NODE":
                node_tables.append(row["name"])
            elif row.get("type") == "REL":
                rel_tables.append(row["name"])

        # Count nodes and relationships
        total_nodes = 0
        total_rels = 0

        stats_table = Table(title="Database Contents")
        stats_table.add_column("Type", style="cyan")
        stats_table.add_column("Count", justify="right", style="magenta")

        for node_type in sorted(node_tables):
            try:
                result = backend.execute(f"MATCH (n:{node_type}) RETURN count(n) as count")
                count = result.get_as_df().iloc[0]["count"]
                total_nodes += count
                stats_table.add_row(f"{node_type} nodes", str(count))
            except Exception:
                pass

        for rel_type in sorted(rel_tables):
            try:
                result = backend.execute(f"MATCH ()-[r:{rel_type}]->() RETURN count(r) as count")
                count = result.get_as_df().iloc[0]["count"]
                total_rels += count
                stats_table.add_row(f"{rel_type} relationships", str(count))
            except Exception:
                pass

        stats_table.add_row("Total nodes", str(total_nodes), style="bold")
        stats_table.add_row("Total relationships", str(total_rels), style="bold")

        console.print(stats_table)
        return {"nodes": total_nodes, "relationships": total_rels}

    except Exception as e:
        console.print(f"[red]❌ Error getting database stats: {e}[/red]")
        return None


# Test if database is available
def check_database():
    """Check if the EKG database is available and populated."""
    try:
        backend = get_backend()
        console.print("[green]✅ EKG database is available[/green]")

        # Get basic stats
        tables_result = backend.execute("CALL show_tables() RETURN *")
        tables_df = tables_result.get_as_df()
        node_count = len([row for _, row in tables_df.iterrows() if row.get("type") == "NODE"])
        rel_count = len([row for _, row in tables_df.iterrows() if row.get("type") == "REL"])
        console.print(f"📊 Contains {node_count} node tables and {rel_count} relationship tables")

        return True
    except Exception as e:
        console.print(f"[red]❌ Error connecting to database: {e}[/red]")
        return False


# Initialize the notebook
console.print(Panel("[bold green]Cypher Query Development Notebook Initialized[/bold green]"))
console.print("[dim]Use the functions defined above to interact with the EKG database[/dim]")

# Check database availability
db_available = check_database()
if not db_available:
    console.print("[yellow]💡 Add data first: [bold]cli kg add --key <data_key>[/bold][/yellow]")

# Show available functions
console.print("\n[bold cyan]Available Functions:[/bold cyan]")
console.print("• execute_cypher_query(query) - Execute raw Cypher queries")
console.print("• execute_text_query(query) - Convert natural language to Cypher")
console.print("• show_schema() - Display graph schema")
console.print("• get_database_stats() - Get database statistics")

2025-12-01 18:48:53.881 | INFO     | genai_graph.core.graph_registry:model_post_init:62 - import genai_graph.ekg.schema.rainbow_review:ReviewedOpportunitySubgraph
2025-12-01 18:48:54.001 | INFO     | genai_graph.core.graph_registry:model_post_init:62 - import genai_graph.ekg.schema.architecture_doc:ArchitectureDocumentSubgraph


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cypher Query Development Notebook Initialized                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Use the functions defined above to interact with the EKG database

✅ EKG database is available

📊 Contains 11 node tables and 11 relationship tables

Available Functions:

• execute_cypher_query(query) - Execute raw Cypher queries

• execute_text_query(query) - Convert natural language to Cypher

• show_schema() - Display graph schema

• get_database_stats() - Get database statistics

## Database Information

Run this cell to get an overview of your database contents and schema.


In [2]:
# Get database statistics
stats = get_database_stats()
if stats:
    print(f"Database contains {stats['nodes']} nodes and {stats['relationships']} relationships")

# Show schema documentation
print("\n" + "=" * 50)
show_schema()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Database Statistics                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

              Database Contents              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                              ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Competitor nodes                  │     4 │
│ Customer nodes                    │     1 │
│ Opportunity nodes                 │     2 │
│ Partner nodes                     │     6 │
│ Person nodes                      │    20 │
│ ReviewedOpportunity nodes         │     2 │
│ RiskAnalysis nodes                │     8 │
│ SWArchitectureDocument nodes      │     1 │
│ Solution nodes                    │     2 │
│ TechnicalApproach nodes           │     2 │
│ TechnicalComponent nodes          │     8 │
│ HAS_COMPETITOR relationships      │     6 │
│ HAS_CONTACT relationships         │     8 │
│ HAS_CUSTOMER relationships        │     3 │
│ HAS_PARTNER relationships         │     6 │
│ HAS_RISK relationships            │     8 │
│ HAS_TEAM_MEMBER relationships     │    13 │
│ HAS_TECH_STACK relationships      │     2 │
│ REVIEWS relationships             │     2 │
│ SOFWARE_ARCHITECURE relationships │     1 │
│ USED_SOLUTION relationships       │     2 │
│ USED_TECHNOLOGY relationships     │     8 │
│ Total nodes                       │    56 │
│ Total relationships               │    59 │
└───────────────────────────────────┴───────┘

Database contains 56 nodes and 59 relationships



╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Knowledge Graph Schema                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<@beartype(genai_graph.core.graph_schema.GraphSchema._validate_coherence) at 0x7ca5794a6b60>:12: UserWarning: Graph schema validation: No field paths found for ReviewedOpportunity in the root model structure; this node may be orphaned.
<@beartype(genai_graph.core.graph_schema.GraphSchema._validate_coherence) at 0x7ca5794a6b60>:12: UserWarning: Graph schema validation: No field paths found for RiskAnalysis in the root model structure; this node may be orphaned.
<@beartype(genai_graph.core.graph_schema.GraphSchema._validate_coherence) at 0x7ca5794a6b60>:12: UserWarning: Graph schema validation: No field paths found for TechnicalApproach in the root model structure; this node may be orphaned.
<@beartype(genai_graph.core.graph_schema.GraphSchema._validate_coherence) at 0x7ca5794a6b60>:12: UserWarning: Graph schema validation: No field paths found for Competitor in the root model structure; this node may be orphaned.
<@beartype(genai_graph.core.graph_schema.GraphSchema._validate_coherence) 

## Graph Schema Description

### Node Types and their fields (labels)

Opportunity // Core opportunity information with financial metrics embedded
  name: string // Official name of the project/opportunity
  opportunity_id: string // Unique identifier from Salesforce or similar CRM
  status: enum(OpportunityStatus)? // Current phase: Pursuit/Active/Completed/Cancelled
  win_loss.result: string // Win/Loss outcome (win|loss|unknown)
  win_loss.reason: string? // Short reason for the outcome

Customer // Customer organization details
  name: string // Client organization name
  segment: string? // Industry vertical of the customer

Person // Individual contacts and team members
  name: string? // Full name of involved person
  organization: string? // Affiliated organization
  contact_type: string? // Internal/External/Client/Partner

SWArchitectureDocument // Root node containing the complete architecture document with technical stack and 
solutions
  document_date: string? // Publication or last-update date in ISO 8601 format YYYY-MM-DD.
  keywords: string[]? // Keywords/tags extracted for search and similarity (e.g., 'microservices', 'Kafka').
  file_metadata.source: string // Source of the file from which the data was extracted

TechnicalComponent // Individual technology, framework, platform, tool, or infrastructure component
  name: string
  type: enum(TechnicalComponentType)

Solution // Specific product, managed service, or OSS solution used in the architecture
  name: string? // Product/solution name (e.g., 'Azure API Management').
  vendor: string? // Vendor/organization providing the solution.
  type: string? // Category (e.g., 'PaaS', 'SaaS', 'Database', 'Message Broker').

ReviewedOpportunity // Root node containing the complete reviewed opportunity
  opportunity: Opportunity
  start_date: string? // Planned/projected start date
  keywords: string[]? // Key terms for semantic search
  team: Person[] // Atos or Eviden person involved
  financials.tcv: float? // Total Contract Value in EUR
  financials.annual_revenue: float? // Yearly revenue projection
  financials.project_margin: float? // Gross margin percentage
  competition.competitive_position: string? // Position vs competition
  competition.differentiators: string[]? // Unique selling points vs competition
  competitors: Competitor[]? // Identified competitors for that opportunity
  partners: Partner[]? // Third-party partners/subcontractors
  file_metadata.source: string // Source of the file from which the data was extracted

RiskAnalysis // Risk assessment and mitigation details
  risk_category: enum(SWProjectRisks)? // classification of the risk

TechnicalApproach // Technical implementation approach and stack
  architecture: string? // General information on architecture
  technical_stack: string[]? // technical stack (platforms, frameworks, third-party softwares, programming 
language, ...)

Competitor // Competitor
  name: string // name of the competitor
  known_as: enum(KnownCompetitor)? // Competitor id (short name)

Partner // Atos partner organization information
  name: string

### Relationships and their properties

Customer → HAS_CONTACT → Person // Customer contact persons
  role: string? // Formal role in project

Opportunity → HAS_CUSTOMER → Customer // Opportunity belongs to customer

ReviewedOpportunity → REVIEWS → Opportunity // Review relationship to core opportunity
ReviewedOpportunity → HAS_TEAM_MEMBER → Person // Internal team members
  role: string? // Formal role in project
ReviewedOpportunity → HAS_PARTNER → Partner // Partner organizations involved
  role: string? // role in the project
ReviewedOpportunity → HAS_RISK → RiskAnalysis // Identified risks and mitigations
  mitigation_strategy: string? // Planned mitigation approach
  risk_description: string // Risk description in the document
ReviewedOpportunity → HAS_TECH_STACK → TechnicalApproach // Technical implementation approach
ReviewedOpportunity → HAS_COMPETITOR → Competitor 

## Basic Cypher Query Examples

These examples demonstrate fundamental Cypher query patterns. Uncomment and run the queries you're interested in.


In [ ]:
# === BASIC CYPHER QUERIES ===

# 1. Count all nodes by type
query1 = "MATCH (n) RETURN labels(n)[0] as node_type, count(n) as count ORDER BY count DESC"
execute_cypher_query(query1)

# 2. Find all ReviewedOpportunity nodes
query2 = "MATCH (n:ReviewedOpportunity) RETURN n.name, n.document_date ORDER BY n.document_date DESC LIMIT 10"
execute_cypher_query(query2)

# 3. Find all Competitor nodes
query3 = "MATCH (c:Competitor) RETURN c.name ORDER BY c.name LIMIT 20"
execute_cypher_query(query3)

# 4. Find all relationship types
query4 = "MATCH ()-[r]->() RETURN type(r) as relationship_type, count(*) as count ORDER BY count DESC"
execute_cypher_query(query4)

# 5. Find nodes with specific properties (case-insensitive)
query5 = "MATCH (n:ReviewedOpportunity) WHERE toLower(n.name) CONTAINS toLower('Venus') RETURN n.name, n.document_date"
execute_cypher_query(query5)

# 6. Find opportunities created after a specific date
query6 = "MATCH (n:ReviewedOpportunity) WHERE date(n.document_date) > date('2020-01-01') RETURN n.name, n.document_date ORDER BY n.document_date DESC"
execute_cypher_query(query6)

## Relationship and Path Queries

These examples show how to query relationships and paths in the graph.


In [ ]:
# === RELATIONSHIP QUERIES ===

# 1. Find opportunities and their competitors
query1 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
RETURN ro.name as opportunity, c.name as competitor
ORDER BY ro.name, c.name
LIMIT 20
"""
execute_cypher_query(query1)

# 2. Find opportunities with customer information
query2 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_CUSTOMER]->(cust:Customer)
RETURN ro.name as opportunity, cust.name as customer, cust.type as customer_type
ORDER BY ro.name
LIMIT 15
"""
execute_cypher_query(query2)

# 3. Find opportunities with partners
query3 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_PARTNER]->(p:Partner)
RETURN ro.name as opportunity, p.name as partner, p.type as partner_type
ORDER BY ro.name
LIMIT 15
"""
execute_cypher_query(query3)

# 4. Find opportunities with risks
query4 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_RISK]->(r:RiskAnalysis)
RETURN ro.name as opportunity, r.risk_level, r.description
ORDER BY ro.name
LIMIT 10
"""
execute_cypher_query(query4)

# 5. Find multi-hop paths (opportunity -> competitor -> opportunity)
query5 = """
MATCH (ro1:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)<-[:HAS_COMPETITOR]-(ro2:ReviewedOpportunity)
WHERE ro1 <> ro2
RETURN ro1.name as opportunity1, c.name as competitor, ro2.name as opportunity2
LIMIT 10
"""
execute_cypher_query(query5)

# 6. Find opportunities with technical approaches
query6 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_TECHNICAL_APPROACH]->(ta:TechnicalApproach)
RETURN ro.name as opportunity, ta.description as technical_approach
LIMIT 10
"""
execute_cypher_query(query6)

## Statistical and Aggregation Queries

These examples demonstrate how to perform statistical analysis and aggregations.


In [ ]:
# === STATISTICAL QUERIES ===

# 1. Financial statistics for opportunities
query1 = """
MATCH (n:ReviewedOpportunity)
RETURN 
  count(n) as total_opportunities,
  avg(n.financials.tcv) as avg_tcv,
  max(n.financials.tcv) as max_tcv,
  min(n.financials.tcv) as min_tcv,
  avg(n.financials.acv) as avg_acv
"""
execute_cypher_query(query1)

# 2. Count opportunities by year
query2 = """
MATCH (n:ReviewedOpportunity)
RETURN 
  substring(n.document_date, 0, 4) as year,
  count(n) as opportunity_count
ORDER BY year DESC
"""
execute_cypher_query(query2)

# 3. Count competitors per opportunity
query3 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
RETURN ro.name as opportunity, count(c) as competitor_count
ORDER BY competitor_count DESC
LIMIT 20
"""
execute_cypher_query(query3)

# 4. Average TCV by number of competitors
query4 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
WITH ro, count(c) as competitor_count
RETURN competitor_count, avg(ro.financials.tcv) as avg_tcv, count(ro) as opportunity_count
ORDER BY competitor_count
"""
execute_cypher_query(query4)

# 5. Risk analysis statistics
query5 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_RISK]->(r:RiskAnalysis)
RETURN r.risk_level, count(*) as risk_count
ORDER BY risk_count DESC
"""
execute_cypher_query(query5)

# 6. Partner type distribution
query6 = """
MATCH (p:Partner)
RETURN p.type as partner_type, count(p) as count
ORDER BY count DESC
"""
execute_cypher_query(query6)

## Advanced Pattern Queries

These examples show more complex query patterns and graph traversals.


In [ ]:
# === ADVANCED PATTERN QUERIES ===

# 1. Find opportunities with high-value competitors
query1 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
WHERE ro.financials.tcv > 1000000
RETURN ro.name as opportunity, ro.financials.tcv as tcv, c.name as competitor
ORDER BY ro.financials.tcv DESC
LIMIT 15
"""
execute_cypher_query(query1)

# 2. Find opportunities with multiple partners and competitors
query2 = """
MATCH (ro:ReviewedOpportunity)
OPTIONAL MATCH (ro)-[:HAS_PARTNER]->(p:Partner)
OPTIONAL MATCH (ro)-[:HAS_COMPETITOR]->(c:Competitor)
WITH ro, count(DISTINCT p) as partner_count, count(DISTINCT c) as competitor_count
WHERE partner_count > 1 AND competitor_count > 1
RETURN ro.name as opportunity, partner_count, competitor_count, ro.financials.tcv as tcv
ORDER BY tcv DESC
"""
execute_cypher_query(query2)

# 3. Find similar opportunities based on shared competitors
query3 = """
MATCH (c:Competitor)<-[:HAS_COMPETITOR]-(ro:ReviewedOpportunity)
WITH c, collect(ro) as opportunities
WHERE size(opportunities) > 2
RETURN c.name as competitor, [o in opportunities | o.name] as opportunity_names
ORDER BY size(opportunities) DESC
"""
execute_cypher_query(query3)

# 4. Find opportunities with specific risk patterns
query4 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_RISK]->(r:RiskAnalysis)
WHERE r.risk_level IN ['HIGH', 'CRITICAL']
WITH ro, collect(r) as risks
WHERE size([r in risks WHERE r.risk_level = 'CRITICAL']) > 0
RETURN ro.name as opportunity, [r in risks | r.description] as risk_descriptions
LIMIT 10
"""
execute_cypher_query(query4)

# 5. Find opportunities with complex technical approaches
query5 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_TECHNICAL_APPROACH]->(ta:TechnicalApproach)
WHERE size(split(ta.description, ' ')) > 50
RETURN ro.name as opportunity, size(split(ta.description, ' ')) as word_count
ORDER BY word_count DESC
LIMIT 10
"""
execute_cypher_query(query5)

# 6. Find opportunities with specific customer types
query6 = """
MATCH (ro:ReviewedOpportunity)-[:HAS_CUSTOMER]->(cust:Customer)
WHERE toLower(cust.type) CONTAINS toLower('government')
RETURN ro.name as opportunity, cust.name as customer, cust.type as customer_type
ORDER BY ro.name
LIMIT 15
"""
execute_cypher_query(query6)

## Text-to-Cypher Examples

These examples demonstrate natural language to Cypher translation. The system will automatically generate Cypher queries from your natural language questions.


In [7]:
# === TEXT-TO-CYPHER EXAMPLES ===

# Simple queries
# query1 = "List all opportunities"
# execute_text_query(query1)

# query2 = "Show me all competitors"
# execute_text_query(query2)

# Filtered queries
# query3 = "List opportunities created after 2020"
# execute_text_query(query3)

# query4 = "Find opportunities with more than 2 competitors"
# execute_text_query(query4)

# Statistical queries
# query5 = "What is the average TCV of all opportunities?"
# execute_text_query(query5)

# query6 = "Which opportunities have the highest TCV?"
# execute_text_query(query6)

# Relationship queries
# query7 = "Show opportunities and their competitors"
# execute_text_query(query7)

# query8 = "List opportunities with government customers"
# execute_text_query(query8)

# Complex queries
# query9 = "Find large opportunities (over $1M) with high-risk profiles"
# execute_text_query(query9)

# query10 = "Show opportunities from 2021 with both partners and competitors"
# execute_text_query(query10)

# Date-based queries
# query11 = "What opportunities were created in the last year?"
# execute_text_query(query11)

# query12 = "Show opportunities with competitors from 2020"
# execute_text_query(query12)


## Custom Query Development

Use this section to develop and test your own queries. Replace the examples with your specific use cases.


In [ ]:
# === CUSTOM QUERY DEVELOPMENT ===

# Replace these examples with your own queries:

# Example 1: Custom Cypher query
# my_cypher_query = """
# MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
# WHERE ro.financials.tcv > 500000 AND c.name CONTAINS 'Tech'
# RETURN ro.name as opportunity, c.name as competitor, ro.financials.tcv as tcv
# ORDER BY ro.financials.tcv DESC
# LIMIT 10
# """
execute_cypher_query(my_cypher_query)

# Example 2: Custom text-to-cypher query
# my_text_query = "Find medium-sized opportunities (between $500K and $2M) with technology competitors"
# execute_text_query(my_text_query)

# Example 3: Complex aggregation query
# my_agg_query = """
# MATCH (ro:ReviewedOpportunity)
# OPTIONAL MATCH (ro)-[:HAS_COMPETITOR]->(c:Competitor)
# OPTIONAL MATCH (ro)-[:HAS_PARTNER]->(p:Partner)
# WITH ro, count(DISTINCT c) as comp_count, count(DISTINCT p) as part_count
# RETURN
#   CASE
#     WHEN ro.financials.tcv < 500000 THEN 'Small'
#     WHEN ro.financials.tcv < 2000000 THEN 'Medium'
#     ELSE 'Large'
#   END as size_category,
#   avg(comp_count) as avg_competitors,
#   avg(part_count) as avg_partners,
#   count(ro) as opportunity_count
# ORDER BY opportunity_count DESC
# """
execute_cypher_query(my_agg_query)

# Add your own queries below:

## Query Performance Analysis

This section helps analyze query performance and optimize slow queries.


In [9]:
# === QUERY PERFORMANCE ANALYSIS ===


def analyze_query_performance(cypher_query, iterations=3):
    """Analyze query performance by running multiple iterations."""
    console.print(Panel("[bold cyan]Query Performance Analysis[/bold cyan]"))
    console.print(f"[dim]Query: {cypher_query}[/dim]")

    times = []
    results = []

    for i in range(iterations):
        start_time = time.time()
        result = execute_cypher_query(cypher_query, show_timing=False)
        execution_time = time.time() - start_time

        times.append(execution_time)
        results.append(result)
        console.print(f"Iteration {i + 1}: {execution_time:.3f} seconds")

    avg_time = sum(times) / len(times)
    min_time = min(times)
    max_time = max(times)

    console.print("\n[bold]Performance Summary:[/bold]")
    console.print(f"Average: {avg_time:.3f} seconds")
    console.print(f"Min: {min_time:.3f} seconds")
    console.print(f"Max: {max_time:.3f} seconds")
    console.print(f"Std Dev: {(sum((t - avg_time) ** 2 for t in times) / len(times)) ** 0.5:.3f} seconds")

    return times


# Example usage:
# test_query = "MATCH (n:ReviewedOpportunity) RETURN count(n)"
# analyze_query_performance(test_query, iterations=5)

# Compare different query approaches:
# query1 = "MATCH (n) RETURN labels(n)[0], count(n)"
# query2 = "CALL show_tables() RETURN name, type"
#
# print("Query 1:")
# times1 = analyze_query_performance(query1, iterations=3)
# print("\nQuery 2:")
# times2 = analyze_query_performance(query2, iterations=3)

## Schema Exploration

Use these queries to explore the graph schema and understand the data structure.


In [ ]:
# === SCHEMA EXPLORATION ===

# 1. List all node labels
query1 = "CALL show_tables() RETURN name, type WHERE type = 'NODE'"
execute_cypher_query(query1)

# 2. List all relationship types
query2 = "CALL show_tables() RETURN name, type WHERE type = 'REL'"
execute_cypher_query(query2)

# 3. Sample properties for a specific node type
query3 = """
MATCH (n:ReviewedOpportunity)
RETURN 
  keys(n) as property_keys,
  n.name as sample_name,
  n.document_date as sample_date
LIMIT 1
"""
execute_cypher_query(query3)

# 4. Find all properties across all nodes of a type
query4 = """
MATCH (n:ReviewedOpportunity)
UNWIND keys(n) as key
RETURN DISTINCT key, count(*) as node_count
ORDER BY node_count DESC
"""
execute_cypher_query(query4)

# 5. Explore relationship properties
query5 = """
MATCH ()-[r:HAS_COMPETITOR]->()
RETURN DISTINCT keys(r) as relationship_properties
LIMIT 1
"""
execute_cypher_query(query5)

# 6. Find orphaned nodes (nodes with no relationships)
query6 = """
MATCH (n)
WHERE NOT (n)--()
RETURN labels(n)[0] as node_type, count(n) as orphaned_count
"""
execute_cypher_query(query6)

## Tips and Best Practices

### Cypher Query Optimization
1. **Use specific labels**: Always specify node labels when possible
2. **Limit results**: Use LIMIT clauses for large result sets
3. **Filter early**: Apply WHERE clauses as early as possible
4. **Use indexes**: Take advantage of indexed properties
5. **Avoid cartesian products**: Be careful with multiple MATCH clauses

### Text-to-Cypher Tips
1. **Be specific**: Clear, specific questions get better results
2. **Use proper terminology**: Reference node types and relationship names from the schema
3. **Include constraints**: Specify date ranges, value limits, etc.
4. **Test iteratively**: Start simple and add complexity gradually

### Debugging Queries
1. **Start simple**: Build complex queries step by step
2. **Check schema**: Use schema exploration queries to understand data structure
3. **Verify data**: Use LIMIT and sampling to verify query logic
4. **Performance test**: Use the performance analysis tools for optimization

### Common Patterns
- **Counting**: `MATCH (n:Label) RETURN count(n)`
- **Filtering**: `MATCH (n) WHERE n.property > value RETURN n`
- **Relationships**: `MATCH (a)-[:REL_TYPE]->(b) RETURN a, b`
- **Aggregation**: `MATCH (n) RETURN n.category, avg(n.value) GROUP BY n.category`
